In [9]:
import pandas as pd
m_df = pd.read_csv('musinsa_data.csv')
musinsa_df = m_df.drop(['c_number','c_name','link','size'], axis=1)

In [10]:
X = musinsa_df[['temp', 'gender', 'height(cm)', 'weight(kg)', 'fit']]
y = musinsa_df['c_result']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True )

In [11]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer

stsc = StandardScaler()
ohe = OneHotEncoder()
mct = make_column_transformer(
    (ohe, ['gender','fit']),
    (stsc, ['height(cm)','weight(kg)','temp'])
)
X_train_transformed = mct.fit_transform(X_train)
X_test_transformed = mct.transform(X_test)

In [12]:
from imblearn.over_sampling import BorderlineSMOTE, SMOTE
n_neighbors = 1  # 또는 더 작은 값으로 설정해도 됩니다
bsmt = SMOTE(sampling_strategy='auto', k_neighbors=n_neighbors)
X_train_transformed_resampled, y_train_resampled = bsmt.fit_resample(X_train_transformed, y_train)

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

params ={
    'n_estimators':[100],
    'max_depth':[6,8,10,12],
}
# rf = RandomForestClassifier(random_state=0, n_jobs=-1, bootstrap=False, )
# grid_cv = GridSearchCV(rf, param_grid=params, cv=2, n_jobs=-1)
# grid_cv.fit(X_train_transformed_resampled, y_train_resampled)

In [21]:
rf = RandomForestClassifier(random_state=0, n_jobs=-1, bootstrap=False, n_estimators=100,max_depth=12)
rf_model = rf.fit(X_train_transformed_resampled, y_train_resampled)

In [22]:
pred = rf_model.predict(X_test_transformed)

In [23]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.3054


In [24]:
import pickle
pickle.dump(rf_model, open('rf.pkl', 'wb'))